# Исследование надежности заемщиков.

### Шаг 1. Обзор данных

In [1]:
import pandas as pd

data = pd.read_csv('/datasets/data.csv')
data.info()
display(data.head(20))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи



### Шаг 2.1 Заполнение пропусков

In [2]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Есть пропущеные значения в столбцах 'days_employed' и 'total_income', это колличественные данные. Их значение может повлиять на результаты исследования  

In [3]:
print('Доля пропущеных значений в столбце days_employed: {:.0%}'.format(len(data[data['days_employed'].isna() == True]) / len(data['days_employed'])))

print('Доля пропущеных значений в столбце total_income: {:.0%}'.format(len(data[data['total_income'].isna() == True]) / len(data['total_income'])))

Доля пропущеных значений в столбце days_employed: 10%
Доля пропущеных значений в столбце total_income: 10%


In [4]:
 #Проверяем  совпадают ли строки с пропусками
data[(data['days_employed'].isna() == True) &(data['total_income'].isna() == True)]['children'].count()

2174

Так как пропущеные значения находтся в колонках "Общий трудовой стаж" и "Ежемесячный доход", а также их количество и строки совпадает вроятнее всего это техническая ошибка.

Не можем заполнить пропуски так-как значения 'days_employed' не корректны. Есть теория что положительные значения стобца 'days_employed' встречаются не у всех значений столбца 'income_type'

In [5]:
#Значения типов занятости для которых общий трудовой стаж выражен положительныс числом
print(data[data['days_employed'] > 0]['income_type'].unique()) 

#Значения типов занятости для которых общий трудовой стаж выражен отрицательным числом
print(data[data['days_employed'] < 0]['income_type'].unique()) 

['пенсионер' 'безработный']
['сотрудник' 'компаньон' 'госслужащий' 'студент' 'предприниматель'
 'в декрете']


Видим что заничения 'days_employed' имеющие не отрицательно значение встречаются только у 'пенсионер' 'безработный'.

In [6]:
#ищем минимальное положительное значение 'days_employed' 
data[data['days_employed'] > 0]['days_employed'].min()

328728.72060451825

Так же положительные знаечения слишком велики, возможна ошибки едениц измерения. Вероятно значение указано в часах, а не в днях.

In [7]:
#конвертируем значения из часов в дни, проверим что получилось
def convert_hours (hours):
    if hours > 0:
        hours = hours / 24
    return hours
data['days_employed'] = data['days_employed'].apply(convert_hours)
data[data['days_employed'] > 0].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
18,0,16678.380705,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,14106.331371,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,15147.853723,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости
30,1,13982.569521,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456.067993,операции с коммерческой недвижимостью
35,0,16417.544674,68,среднее,1,гражданский брак,1,M,пенсионер,0,77805.677436,на проведение свадьбы
50,0,14738.809681,63,среднее,1,женат / замужем,0,F,пенсионер,0,92342.730612,автомобили
56,0,15422.711968,64,среднее,1,вдовец / вдова,2,F,пенсионер,0,149141.043533,образование
71,0,14088.063746,62,среднее,1,женат / замужем,0,F,пенсионер,0,43929.696397,автомобили
78,0,14988.456045,61,высшее,0,женат / замужем,0,M,пенсионер,0,175127.646000,сделка с автомобилем


Теперь значения похожи на реальные данные в днях. Также требуется заменить отрицательные значения в столбце 'days_employed' 

In [8]:
data['days_employed'] = data['days_employed'].apply(abs)
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Теперь заменим пропуски медианным значением по столбцу, в столбцах 'days_employed' 'total_income'. Медианное значение подходит лучше так как среднее значение является слишком чувствительным к так называемым «выбросам»

In [9]:
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median())
data['total_income'] = data['total_income'].fillna(data['total_income'].median())
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Шаг 2.2 Проверка данных на аномалии и исправления.

In [10]:
print(data['children'].min(), data['children'].max()) #Проверяем максимальные и минимальные значения столбца "children"

-1 20


В столбце 'children' есть отрицательные значения чего быть не может, скорей всего это ошибка ввода. максимальное значение 20 а это уже может быть как ошибкой так и вполне реальным значением, оставим как есть.

In [11]:
data['children'] = data['children'].apply(abs) #Преобразуем все значения в положительные

### Шаг 2.3. Изменение типов данных.

Заменим вещественный тип данных в столбцах 'total_income' и 'days_employed' на целочисленный

In [12]:
data['total_income'] = data['total_income'].astype('int')
data['days_employed'] = data['days_employed'].astype('int')
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


### Шаг 2.4. Удаление дубликатов.

Сначала проверим на неявные дубликаты и исправим их, и только затем проверим на явные так как количество явных дубликатов может увеличится. 

In [13]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Имеются записи с разным регистром, приведём их к единому виду.

In [14]:
data['education'] = data['education'].str.lower()
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [15]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [16]:
data['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

In [17]:
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [18]:
data['purpose'].sort_values().unique()

array(['автомобили', 'автомобиль', 'высшее образование',
       'дополнительное образование', 'жилье',
       'заняться высшим образованием', 'заняться образованием',
       'на покупку автомобиля', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля', 'на проведение свадьбы',
       'недвижимость', 'образование', 'операции с жильем',
       'операции с коммерческой недвижимостью',
       'операции с недвижимостью', 'операции со своей недвижимостью',
       'покупка жилой недвижимости', 'покупка жилья',
       'покупка жилья для сдачи', 'покупка жилья для семьи',
       'покупка коммерческой недвижимости', 'покупка недвижимости',
       'покупка своего жилья', 'получение высшего образования',
       'получение дополнительного образования', 'получение образования',
       'приобретение автомобиля', 'профильное образование',
       'ремонт жилью', 'свадьба', 'свой автомобиль',
       'сделка с автомобилем', 'сделка с подержанным автомобилем',
       'строительство 

В столбце 'purpose' есть неявные дубликаты, судя по всему данные вводились вручную. Хорошо бы сделать ввод данных из предопределённых значений. Так же в задании далее указано "Создайте функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category", так что пока что оставим без изменений.

Далее проверим данные на явные дубликаты.

In [19]:
print('Обнаружено', data.duplicated().sum(), 'строчек с дублированными значениями')

Обнаружено 71 строчек с дублированными значениями


 Удалим их, и проверим.

In [20]:
data = data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()

0

Возможными причинами дубликатов могли стать: человеческий фактор, сбои в работе программного обеспечения в котором эти данные вводились, не корректная выгрузка. 

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создадим отдельный датафрэйм с колонками 'education' 'family_statu' 'education_id' 'family_status_id', и уберём их из исодного датафрэйма и оставим только идентификаторы 'education_id' 'family_status_id'.

In [21]:

# создаём словари с идентификаторами
dict_education = data[['education_id','education']]
dict_family_statu = data[['family_status_id','family_status']]
# убираем дублированные значения
dict_education = dict_education.drop_duplicates().reset_index(drop=True)
dict_family_statu = dict_family_statu.drop_duplicates().reset_index(drop=True)
#убираме столбцы из основного датафрэйма
data = data.drop(columns = ['education', 'family_status'], axis = 1) 

display(dict_education)
display(dict_family_statu)
data.head(10)

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177,53,1,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,0,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,0,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,1,0,M,сотрудник,0,135823,образование
8,2,6929,35,0,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи


### Шаг 2.6. Категоризация дохода.

Добавим столбец с категориями доходов для более удобной работы и наглядности.

In [22]:
# Функция для деления дохода по категориям в зависмости от заначения столбца 'total_income'
def total_income_category(income):
    if income <= 30000:
        return 'E'
    if income <= 50000:
        return 'D'
    if income <= 200000:
        return 'C'
    if income <= 1000000:
        return 'B'
    return 'A'

# Применяем фунцию
data['total_income_category'] = data['total_income'].apply(total_income_category)

data.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,14177,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C
5,0,926,27,0,1,M,компаньон,0,255763,покупка жилья,B
6,0,2879,43,0,0,F,компаньон,0,240525,операции с жильем,B
7,0,152,50,1,0,M,сотрудник,0,135823,образование,C
8,2,6929,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C
9,0,2188,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C


### Шаг 2.7. Категоризация целей кредита.

Сделаем ещё одну колонку 'purpose_category' в которую запишем в зависимости от значений столбца 'purpose' категории: 
'операции с автомобилем'
'операции с недвижимостью'
'проведение свадьбы'
'получение образования'

In [23]:
# Создаём колонку 'purpose_category' идентичную колонке 'purpose', затем в не функцией replace() заменим значения на нужные
data['purpose_category'] = data['purpose']
data['purpose_category'] = data['purpose_category'].replace(['автомобили','на покупку автомобиля',
                                           'на покупку подержанного автомобиля',
                                           'на покупку своего автомобиля',
                                           'приобретение автомобиля,',
                                           'автомобиль',
                                           'свой автомобиль','сделка с автомобилем',
                                           'сделка с подержанным автомобилем',
                                           'приобретение автомобиля'],'операции с автомобилем')

data['purpose_category'] = data['purpose_category'].replace(['высшее образование',
                                           'дополнительное образование',
                                           'заняться высшим образованием',
                                           'заняться образованием',
                                           'образование',
                                           'получение высшего образования',
                                           'получение дополнительного образования',
                                           'профильное образование'],'получение образования')

data['purpose_category'] = data['purpose_category'].replace(['жилье',
                                           'операции с жильем',
                                           'операции с коммерческой недвижимостью',
                                           'операции со своей недвижимостью',
                                           'покупка жилой недвижимости',
                                           'покупка жилья',
                                           'покупка жилья для сдачи',
                                           'ремонт жилью',
                                           'ремонт',
                                           'покупка недвижимости',
                                           'покупка жилья для семьи', 
                                           'покупка коммерческой недвижимости',
                                           'покупка своего жилья',
                                           'недвижимость',
                                           'строительство жилой недвижимости',
                                           'строительство собственной недвижимости',
                                           'строительство недвижимости'],'операции с недвижимостью')

data['purpose_category'] = data['purpose_category'].replace(['на проведение свадьбы',
                                           'сыграть свадьбу',
                                           'свадьба'],'проведение свадьбы')
print(data['purpose_category'].sort_values().unique())
data.head(10)


   

['операции с автомобилем' 'операции с недвижимостью'
 'получение образования' 'проведение свадьбы']


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,14177,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926,27,0,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879,43,0,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152,50,1,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,6929,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


### Ответы на вопросы.

Есть ли зависимость между количеством детей и возвратом кредита в срок?

Категоризируем данные по кличеству детей
1) без детей
2) 1-2 ребёнка
3) многодетные

In [24]:
# Функция для деления на котегории в зависмости от значиний столбца 'children'
def children_category(children):
    if children == 0:
        return 'без детей'
    if children <= 2:
        return '1-2 ребёнка'
    return 'многодетные'
# Применяем функцию 
data['children_category'] = data['children'].apply(children_category)
data.head(5)


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category,children_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью,1-2 ребёнка
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем,1-2 ребёнка
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью,без детей
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования,многодетные
4,0,14177,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы,без детей


Теперь сделаем сводную таблцу, в ней отразим процент просрочки по 'children_category'

In [25]:
# Формируем сводную таблицу
children_category_debt = data.pivot_table(index = ['children_category'], values='debt', aggfunc = ('mean', lambda X:X.count()))

# Форматируем значения, имена колонок и отсортируем  для удобства и наглядности
children_category_debt['mean'] = round(children_category_debt['mean']*100, 1)
children_category_debt = children_category_debt.rename(columns={'<lambda_0>':'Общее количество', 'mean':'Процент должников'})

children_category_debt.sort_values('Процент должников')

,Общее количество,Процент должников
children_category,,
без детей,14091.0,7.5
многодетные,456.0,8.6
1-2 ребёнка,6907.0,9.3


Видим что у бездетных самый маленький процент просрочки, так же видем что у многодетных процент просрочки ниже чем у тех у кого 1 или 2 ребёнка. Нульзя утверждать что количество детей значительно влияет на погашение кредита.

Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [26]:
# Формируем сводную таблицу
family_status_debt = data.pivot_table(index = ['family_status_id'], values='debt', aggfunc = ('mean', lambda X:X.count()))

# Форматируем значения и имена колонок 
family_status_debt['mean'] = round(family_status_debt['mean']*100, 1)
family_status_debt = family_status_debt.rename(columns={'<lambda_0>':'Общее количество', 'mean':'Процент должников'})

# Соеденим с словарём dict_family_statu для удобства, и отсартирум для наглядности 
family_status_debt = family_status_debt.merge(dict_family_statu, on='family_status_id', how='left')
family_status_debt.sort_values('Процент должников')

,family_status_id,Общее количество,Процент должников,family_status
2,2,959.0,6.6,вдовец / вдова
3,3,1195.0,7.1,в разводе
0,0,12339.0,7.5,женат / замужем
1,1,4151.0,9.3,гражданский брак
4,4,2810.0,9.8,Не женат / не замужем


Из представленной таблицы видим что процент должников больше всего у людей никогда не состоявших в официальном браке. В тоже время люди в разводе или потерявшие своих супрогов более ответсвенно отностятся у погошению кредитов.

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [27]:
# Формируем сводную таблицу
income_category_debt = data.pivot_table(index = ['total_income_category'], values='debt', aggfunc = ('mean', lambda X:X.count()))
# Форматируем значения, имена колонок и отсортируем  для удобства и наглядности
income_category_debt['mean'] = round(income_category_debt['mean']*100, 1)
income_category_debt = income_category_debt.rename(columns={'<lambda_0>':'Общее количество', 'mean':'Процент должников'})

income_category_debt.sort_values('total_income_category')

,Общее количество,Процент должников
total_income_category,,
A,25.0,8.0
B,5041.0,7.1
C,16016.0,8.5
D,350.0,6.0
E,22.0,9.1


Видим что уровень дохода никак не влияет на на погошение кридита.

In [28]:
Как разные цели кредита влияют на его возврат в срок?

Object `срок` not found.


In [ ]:
Как разные цели кредита влияют на его возврат в срок

In [29]:
# Формируем сводную таблицу. Наверное надо было написать одну функцию и пользоваться ей а не писать одно и тоже 
purpose_category_debt = data.pivot_table(index = ['purpose_category'], values='debt', aggfunc = ('mean', lambda X:X.count()))

# Форматируем значения, имена колонок и отсортируем  для удобства и наглядности
purpose_category_debt['mean'] = round(purpose_category_debt['mean']*100, 1)
purpose_category_debt = purpose_category_debt.rename(columns={'<lambda_0>':'Общее количество', 'mean':'Процент должников'})


purpose_category_debt.sort_values('Процент должников')

,Общее количество,Процент должников
purpose_category,,
операции с недвижимостью,10811.0,7.2
проведение свадьбы,2324.0,8.0
получение образования,4013.0,9.2
операции с автомобилем,4306.0,9.4


Кредты на операции с недвижимостью показывают наименьший процент должников, в то время как на получение образования и операции с автомобилем процент занчительно выше, значения отличаются на ~30%  

## Общий вывод:

Бездетные чаще платят по кредитам в срок, однако нельзя говорить о том что количество детей влияет на платежи. Так же есть зависимость между семейным положением и возвратом кредита в срок. Ответственнее всего к выплатам подходят люди бывавшие в официальном браке или же находяшееся в нём в данный момент, в отличии от никогда не бывавших в офицально браке. И стоит обратить внимание что кредиты на образование и автомобиле имеют большой процент должников.